# Problem - *Rankine cycle with a mixing heat exchanger and with reheat*

## Description:

![priklad](../obrazky/rc_smesovaci_prihrev.jpg)

In [1]:
from pyfluids import Fluid, FluidsList, Input

p1 = p9 = 8_000_000 # Pa
p2 = p7 = p8 = 2_000_000 # Pa
p3 = p4 = 700_000 # Pa
p5 = p6 = 8_000 # MPa

t1 = 480 + 273.15 # K
t4 = 440 + 273.15 # K

x4 = x6 = 0

n_ist = 0.85
n_isp = 0.9

w_expected = 100_000_000 # W

Water = Fluid(FluidsList.Water)

## Prerequisities:


### 1st Law of ThermoDynaics for open systems
    
$$ \frac{dE_{CV}}{dt} = Q_{CV} + \sum\limits_{in}\dot{m}_{in} \cdot h_{tot,in} - \dot{W}_{CV}  - \sum\limits_{out}\dot{m}_{out} \cdot h_{tot,out} $$

$$ h_{tot} = h + \frac{1}{2}c^2 + g \cdot z $$   

- isentropic - $ dS = 0 \Rightarrow Q_{CV} = 0 $

- steady-flow - $ \frac{dE_{CV}}{dt} = 0 $ 

- $ \Delta E_p = 0, \Delta E_k = 0 \Rightarrow h_{tot} = h $

### Isentropic efficiency

1. Turbine

$$ \eta_{is_T} = \frac{(\frac{\dot{W}_t}{\dot{m}})}{(\frac{\dot{W}_t}{\dot{m}})_s} = \frac{h_1-h_2}{h_1-h_{2s}} < 1 $$

2. Pump

$$ \eta_{is_P} = \frac{(\frac{\dot{W}_t}{\dot{m}})_s}{(\frac{\dot{W}_t}{\dot{m}})} = \frac{h_{4s}-h_3}{h_4-h_3} < 1 $$

$$ \eta_{is_P} = \frac{\nu_3\cdot(p_4-p_3)}{h_4-h_3} < 1 $$

## Solution:

### States:

![diagram](../obrazky/diagram_rc_smesovaci_prihrev.jpg)

In [2]:
# Turbine 1 inlet
state1 = Water.with_state(Input.pressure(p1), Input.temperature(t1))

# Turbine 1 outlet
state2s = state1.isentropic_expansion_to_pressure(p2)
h2 = state1.enthalpy - n_ist * (state1.enthalpy - state2s.enthalpy)
state2 = Water.with_state(Input.pressure(p2), Input.enthalpy(h2))

# Turbine 2 outlet
state3s = state2.isentropic_expansion_to_pressure(p3)
h3 = state2.enthalpy - n_ist * (state2.enthalpy - state3s.enthalpy)
state3 = Water.with_state(Input.pressure(p3), Input.enthalpy(h3))

# Turbine 3 inlet
state4 = Water.with_state(Input.pressure(p4), Input.temperature(t4))

# Turbine 3 outlet
state5s = state4.isentropic_expansion_to_pressure(p5)
h5 = state4.enthalpy - n_ist * (state4.enthalpy - state5s.enthalpy)
state5 = Water.with_state(Input.pressure(p5), Input.enthalpy(h5))

# Compressor inlet
state6 = Water.with_state(Input.pressure(p6), Input.quality(0))

# Compressor 1 outlet
h7 = state6.enthalpy + state6.specific_volume * (p7 - p6) / n_isp
state7 = Water.with_state(Input.pressure(p7), Input.enthalpy(h7))

# Compressor 2 inlet
state8 = Water.bubble_point_at_pressure(p8)

# Compressor 2 outlet
h9 = state8.enthalpy + state8.specific_volume * (p9 - p8) / n_isp
state9 = Water.with_state(Input.pressure(p9), Input.enthalpy(h9))

In [3]:
import pandas as pd
from pandas import DataFrame

cycle_states = [state1, state2, state3, state4, state5, state6, state7, state8, state9]
df = DataFrame( [ ( state.pressure, state.temperature, state.specific_volume, state.enthalpy, state.entropy,) for state in cycle_states ],
    columns=[ "Pressure (Pa)", "Temperature (K)", "Specific volume (m^3/kg)", "Enthalpy (J/kg)", "Entropy (J/kgK)", ],
    index=["1", "2", "3", "4", "5", "6", "7", "8", "9"],
)

pd.set_option('display.expand_frame_repr', False)  
print(df)

   Pressure (Pa)  Temperature (K)  Specific volume (m^3/kg)  Enthalpy (J/kg)  Entropy (J/kgK)
1        8000000       753.150000                  0.040362     3.349645e+06      6661.263744
2        2000000       572.198478                  0.125251     3.021973e+06      6764.558611
3         700000       463.459536                  0.292689     2.823296e+06      6841.469484
4         700000       713.150000                  0.466761     3.353892e+06      7758.980883
5           8000       314.658765                 18.034018     2.567597e+06      8199.960312
6           8000       314.658765                  0.001008     1.738398e+05       592.491156
7        2000000       314.772541                  0.001008     1.760719e+05       593.203062
8        2000000       485.527225                  0.001177     9.084981e+05      2446.754200
9        8000000       486.810512                  0.001172     9.163431e+05      2448.396324


### Mixing heat exchanger - y ratio

1. Mass conservation in first turbine

    $$ \dot{m}_1 = \dot{m}_2 + \dot{m}_3 $$

    $$ 1 = \frac{\dot{m}_2}{\dot{m}_1} + \frac{\dot{m}_3}{\dot{m}_1} = y + (1 - y) $$

2. 1st Law of ThermoDynamics

    $$ 0 = \dot{m}_2 \cdot h_2 + \dot{m}_3 \cdot h_7 - \dot{m}_1 \cdot h_8 $$

    $$ 0 = y \cdot h_2 + (1-y) \cdot h_7 - 1 \cdot h_8 $$

    $$ 0 = y \cdot (h_2 - h_7) + (h_7 - h_8) $$

    $$ y = \frac{h_8 - h_7}{h_2 - h_7} $$


In [4]:
y = (state8.enthalpy - state7.enthalpy) / (state2.enthalpy - state7.enthalpy)

### Turbine 

- 1st Law of ThermoDynamics
   
1. First turbine

    $$ w_{o12} = h_1 - h_2 $$

2. Second turbine

    $$ w_{o23} = (1-y) \cdot (h_2 - h_3) $$

3. Third turbine

    $$ w_{o45} = (1-y) \cdot (h_4 - h_5) $$


In [5]:
w_o12 = state1.enthalpy - state2.enthalpy
w_o23 = (1-y)*(state2.enthalpy - state3.enthalpy)
w_o45 = (1-y)*(state4.enthalpy - state5.enthalpy)
w_o = w_o12 + w_o23 + w_o45

### Pumps

- 1st Law of ThermoDynamics

1. First pump

$$ 0 = \dot{m}_3 \cdot (h_6 - h_7) - \dot{W} $$

$$ \frac{\dot{W}}{\dot{m}_1} = w_{i67} = (1-y) \cdot (h_6 - h_7) $$

2. Second pump

$$ 0 = \dot{m}_1 \cdot (h_8 - h_9) - \dot{W} $$

$$ \frac{\dot{W}}{\dot{m}_1} = w_{i89} = h_8 - h_9 $$


In [6]:
w_i67 = (1-y)*(state6.enthalpy - state7.enthalpy)
w_i89 = state8.enthalpy - state9.enthalpy
w_i = w_i67 + w_i89

### Boiler

- 1st Law of ThermoDynamics

1. Primary heating

$$ 0 = \dot{m}_1 \cdot (h_9 - h_1) + \dot{Q}_{19} $$

$$  q_{i19} = h_1 - h_9 $$

2. Reheat

$$ 0 = \dot{m}_3 \cdot (h_3 - h_4) + \dot{Q}_{34} $$

$$  q_{i34} = (1-y) \cdot (h_4 - h_3) $$

In [7]:
q_i19 = state1.enthalpy - state9.enthalpy
q_i34 = (1-y)*(state4.enthalpy - state3.enthalpy)
q_i = q_i19 + q_i34

## Results:

In [8]:
w_total = w_o - abs(w_i)

m_flow = w_expected / w_total

# Thermal efficiency
ny = w_total / (q_i)

print(f'Turbine work: {w_o/1000:.2f} kJ/kg')
print(f'Pump work: {w_i/1000:.2f} kJ/kg')
print(f'Heat added in boiler: {q_i/1000:.2f} kJ/kg')
print(f'Total work: {w_total/1000:.2f} kJ/kg')
print(f'Efficiency: {ny:.4f}')
print(f'Mass flow: {m_flow:.2f} kg/s')


Turbine work: 1059.15 kJ/kg
Pump work: -9.50 kJ/kg
Heat added in boiler: 2827.34 kJ/kg
Total work: 1049.65 kJ/kg
Efficiency: 0.3712
Mass flow: 95.27 kg/s


## T-S Diagram of Rankin Cycle

In [12]:
from math import e
import plotly.graph_objects as go
import numpy as np

state9bp = Water.bubble_point_at_pressure(state9.pressure)
state9dp = Water.dew_point_at_pressure(state9.pressure)
state2dp = Water.dew_point_at_pressure(state2.pressure)

cycle_states = [state1, state2, state3, state4, state5, state6, state7, state8, state9, state9bp, state9dp, state1] 
exchanger_states = [state2, state2dp, state8]

cycle_temperatures = [state.temperature for state in cycle_states]
cycle_entropies = [state.entropy for state in cycle_states]

exchanger_temperatures = [state.temperature for state in exchanger_states]
exchanger_entropies = [state.entropy for state in exchanger_states]

# Plot the saturation lines
t_range = np.linspace(Water.triple_temperature, Water.critical_temperature, 1000)
entropy_saturated_liquid = [Water.bubble_point_at_temperature(t).entropy for t in t_range]
entropy_saturated_vapor = [Water.dew_point_at_temperature(t).entropy for t in t_range]

# Create a trace for liquid saturation line
trace0 = go.Scatter(x=entropy_saturated_liquid, y=t_range, mode="lines", name="Liquid Saturation Line")

# Create a trace for vapor saturation line
trace1 = go.Scatter(x=entropy_saturated_vapor, y=t_range, mode="lines", name="Vapor Saturation Line")

# Create a trace for the cycle
trace2 = go.Scatter(x=cycle_entropies, y=cycle_temperatures, mode="lines+markers", name="Cycle")
trace3 = go.Scatter(x=exchanger_entropies, y=exchanger_temperatures, mode="lines+markers", name="Exchanger")

data = [trace0, trace1, trace2, trace3]

layout = go.Layout(
    title="T-S Diagram",
    xaxis=dict(title="Entropy (J/kg*K)"),
    yaxis=dict(title="Temperature (K)"),
)

fig = go.Figure(data=data, layout=layout)

fig.show()